In [0]:
# COMMAND ---------- (cell 1) Maintenance: OPTIMIZE + Z-ORDER

# Compact small files and cluster by location for faster queries
spark.sql("""
OPTIMIZE weather_silver.daily
ZORDER BY (location_lat, location_lon)
""")

# COMMAND ---------- (cell 2) Maintenance: VACUUM Old Files

# Remove old snapshot files older than 7 days (safe for production)
spark.sql("""
VACUUM weather_silver.daily RETAIN 168 HOURS
""")

# COMMAND ---------- (cell 3) Gold Table: Weekly Aggregates

df_gold_weekly = spark.sql("""
SELECT
  date_trunc('week', date) AS week_start,
  location_lat,
  location_lon,
  ROUND(AVG(avg_temp_c), 2) AS weekly_avg_temp,
  MAX(max_wind_kmh) AS weekly_max_wind,
  ROUND(AVG(min_humidity_pct), 2) AS weekly_avg_humidity,
  COUNT(*) AS days_covered,
  current_timestamp() AS process_ts
FROM weather_silver.daily
GROUP BY 1, 2, 3
""")

spark.sql("CREATE DATABASE IF NOT EXISTS weather_gold")

(df_gold_weekly.write
    .format("delta")
    .mode("overwrite")
    .partitionBy("week_start")
    .saveAsTable("weather_gold.weekly"))

# COMMAND ---------- (cell 4) Gold Table: Monthly Aggregates

df_gold_monthly = spark.sql("""
SELECT
  date_trunc('month', date) AS month_start,
  location_lat,
  location_lon,
  ROUND(AVG(avg_temp_c), 2) AS monthly_avg_temp,
  MAX(max_wind_kmh) AS monthly_max_wind,
  ROUND(AVG(min_humidity_pct), 2) AS monthly_avg_humidity,
  COUNT(*) AS days_covered,
  current_timestamp() AS process_ts
FROM weather_silver.daily
GROUP BY 1, 2, 3
""")

(df_gold_monthly.write
    .format("delta")
    .mode("overwrite")
    .partitionBy("month_start")
    .saveAsTable("weather_gold.monthly"))

# COMMAND ---------- (cell 5) Quick Checks

print("Weekly Gold Preview:")
spark.sql("""
SELECT * FROM weather_gold.weekly
ORDER BY week_start DESC
LIMIT 5
""").show()

print("Monthly Gold Preview:")
spark.sql("""
SELECT * FROM weather_gold.monthly
ORDER BY month_start DESC
LIMIT 5
""").show()
